In [1]:
import re
import pickle
from IPython.display import display

import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, Pipeline

In [2]:
def convert(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

In [4]:
df = pd.read_csv('input/train_V2.csv')
df.columns = [convert(col) for col in df.columns]
display(df.head(), df.shape, list(df.columns))

,id,group_id,match_id,assists,boosts,damage_dealt,dbn_os,headshot_kills,heals,kill_place,...,revives,ride_distance,road_kills,swim_distance,team_kills,vehicle_destroys,walk_distance,weapons_acquired,win_points,win_place_perc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,...,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,...,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,...,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,...,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,...,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


(4446966, 29)

['id',
 'group_id',
 'match_id',
 'assists',
 'boosts',
 'damage_dealt',
 'dbn_os',
 'headshot_kills',
 'heals',
 'kill_place',
 'kill_points',
 'kills',
 'kill_streaks',
 'longest_kill',
 'match_duration',
 'match_type',
 'max_place',
 'num_groups',
 'rank_points',
 'revives',
 'ride_distance',
 'road_kills',
 'swim_distance',
 'team_kills',
 'vehicle_destroys',
 'walk_distance',
 'weapons_acquired',
 'win_points',
 'win_place_perc']

In [5]:
id_features = ['id', 'group_id', 'match_id']
categorical_features = ['match_type', ]
target_feature = 'win_place_perc'
selected_features = [col for col in df.columns if col not in id_features + categorical_features + [target_feature]] 

Drop NaN target

In [6]:
df = df[df['win_place_perc'].notnull()]

Если `SimpleFeatureGenerator` определен в тетрадке, то `n_jobs=-1` в `FeatureUnion` упадет с `pickle error`

In [7]:
from features import SimpleFeatureGenerator, ColumnsSelector
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

feature_generator = FeatureUnion(
    n_jobs=1,
    transformer_list=[
        ('kernel', SimpleFeatureGenerator()),
        ('selected', ColumnsSelector(selected_features)),
    ]
)

pipeline = Pipeline([
    ('feature_generator', feature_generator),
#     ('imputer', SimpleImputer(strategy='median')),
    ('imputer', SimpleImputer(strategy='constant', fill_value=0))
])

In [8]:
# x = pd.DataFrame(pipeline.fit_transform(df), columns=pipeline.named_steps['feature_generator'].get_feature_names())

In [11]:
%%time
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

kfold = KFold(n_splits = 2)
kfold.get_n_splits(df)
score = []
sample_size = 100000
for train_index, test_index in kfold.split(df):
    model = RandomForestRegressor(n_estimators=100, n_jobs=-1)
    x_train = pd.DataFrame(pipeline.fit_transform(df.loc[train_index, :])).fillna(0).sample(sample_size, random_state=12345)
    y_train = pd.DataFrame(df.loc[train_index, target_feature]).fillna(0).sample(sample_size, random_state=12345)
    model.fit(x_train, y_train)
    
    x_test = pd.DataFrame(pipeline.fit_transform(df.loc[test_index, :])).fillna(0).sample(sample_size, random_state=12345)
    y_test = pd.DataFrame(df.loc[test_index, target_feature]).fillna(0).sample(sample_size, random_state=12345)
    step_score = mean_absolute_error(y_test, model.predict(x_test))
    print(step_score)
    score.append(step_score)

/Users/dmitrymikhailovich_/cnt/pubg-kaggle/.direnv/python-3.6.3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
/Users/dmitrymikhailovich_/cnt/pubg-kaggle/.direnv/python-3.6.3/lib/python3.6/site-packages/pandas/core/indexing.py:1027: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]
/Users/dmitrymikhailovich_/cnt/pubg-kaggle/.direnv/python-3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DataConversionWarning:

0.06241621890090759


/Users/dmitrymikhailovich_/cnt/pubg-kaggle/.direnv/python-3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/Users/dmitrymikhailovich_/cnt/pubg-kaggle/.direnv/python-3.6.3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
/Users/dmitrymikhailovich_/cnt/pubg-kaggle/.direnv/python-3.6.3/lib/python3.6/site-packages/pandas/core/indexing.py:1027: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:


0.06217977257688892
CPU times: user 5min 52s, sys: 19.9 s, total: 6min 11s
Wall time: 2min 25s
